<a href="https://colab.research.google.com/github/alfanugraha/mlzoomcamp-homework/blob/master/09_serverless_hw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dataset

This work is based on the previous [one](https://github.com/DataTalksClub/machine-learning-zoomcamp/blob/master/cohorts/2025/08-deep-learning/homework.md). Need to download the data here:

- https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/hair_classifier_v1.onnx.data
- https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/hair_classifier_v1.onnx

In [1]:
!pip install keras-image-helper

In [4]:
!pip install tf2onnx onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.0 MB/s eta 0:00:00


In [5]:
import numpy as np
import onnxruntime as ort
from keras_image_helper import create_preprocessor

In [6]:
def preprocess_pytorch(X):
    # X: shape (1, 299, 299, 3), dtype=float32, values in [0, 255]
    X = X / 255.0

    mean = np.array([0.485, 0.456, 0.406]).reshape(1, 3, 1, 1)
    std = np.array([0.229, 0.224, 0.225]).reshape(1, 3, 1, 1)

    # Convert NHWC → NCHW
    # from (batch, height, width, channels) → (batch, channels, height, width)
    X = X.transpose(0, 3, 1, 2)

    # Normalize
    X = (X - mean) / std

    return X.astype(np.float32)


preprocessor = create_preprocessor(preprocess_pytorch, target_size=(224, 224))

In [14]:
onnx_model_path = "/content/hair_classifier_v1.onnx"
session = ort.InferenceSession(onnx_model_path, providers=["CPUExecutionProvider"])

Fail: [ONNXRuntimeError] : 1 : FAIL : Deserialize tensor fc1.weight failed.tensorprotoutils.cc:1139 GetExtDataFromTensorProto External initializer: fc1.weight offset: 65536 size to read: 80289792 given file_length: 35651584 are out of bounds or can not be read in full.

## Q1. Output node name

## Q2. Image size

## Q3. First R value after pre-processing

## Q4. Model output

## Q5. Docker image size

## Q6. Model output docker